[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_upT_toWSsxcWBuk8TKLLddQoogKaZWjlsI3YRcSGz-n8ZrMgBQrQII6BdTXbo7fSZUg1YhAZitlVJR3s9qgMZbOfqdil1u6h2YYidVwiQNdWMSWSjrMGDCh2cO9gM22oOrWczdDG5r5l_Uokj3eKAvMeBehYbldbG_1vkoHPkw?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_upT_toWSsxcWBuk8TKLLddQoogKaZWjlsI3YRcSGz-n8ZrMgBQrQII6BdTXbo7fSZUg1YhAZitlVJR3s9qgMZbOfqdil1u6h2YYidVwiQNdWMSWSjrMGDCh2cO9gM22oOrWczdDG5r5l_Uokj3eKAvMeBehYbldbG_1vkoHPkw)

In [1]:
%%capture
! pip install git+https://github.com/openai/swarm.git
! pip install pandas_ta

In [2]:
import yfinance as yf
from swarm import Swarm, Agent
import pandas_ta as ta
import pandas as pd
import os

In [3]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [ ]:
# 股價走勢查詢函數
def get_stock_price_history(symbol, period = "1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return history.tail().to_dict('records')
    except:
        return f"無法獲取 {symbol} 的歷史價格"

In [ ]:
get_stock_price_history('2330.TW')

In [ ]:
# 股票資訊查詢函數
def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        return {
            '公司名稱': info.get('longName', 'N/A'),
            '現價': info.get('currentPrice', 'N/A'),
            '市值': info.get('marketCap', 'N/A'),
            '產業': info.get('industry', 'N/A'),
            '52週高點': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52週低點': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except:
        return f"無法獲取 {symbol} 的資訊"

In [ ]:
get_stock_info('2330.TW')

In [ ]:
# 基本財務數據查詢函數
def get_financial_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        # 獲取最近的財務報表
        financials = stock.financials.iloc[:, 0]
        balance_sheet = stock.balance_sheet.iloc[:, 0]
        return {
            '營收': financials.get('Total Revenue', 'N/A'),
            '淨利': financials.get('Net Income', 'N/A'),
            '總資產': balance_sheet.get('Total Assets', 'N/A'),
            '總負債': balance_sheet.get('Total Liabilities Net Minority Interest', 'N/A'),
            'ROE': stock.info.get('returnOnEquity', 'N/A'),
            '本益比': stock.info.get('trailingPE', 'N/A')
        }
    except:
        return f"無法獲取 {symbol} 的財務數據"

In [ ]:
get_financial_data('2330.TW')

In [ ]:
# 技術分析函數
def get_technical_analysis(symbol, period= "6mo"):
    try:
        # 獲取歷史數據
        stock = yf.Ticker(symbol)
        df = stock.history(period=period)

        # 計算技術指標
        # RSI
        df['RSI'] = ta.rsi(df['Close'], length=14)

        # MACD
        macd = ta.macd(df['Close'])
        df = pd.concat([df, macd], axis=1)

        # 移動平均線
        df['SMA_20'] = ta.sma(df['Close'], length=20)
        df['SMA_60'] = ta.sma(df['Close'], length=60)

        # KD指標
        stoch = ta.stoch(df['High'], df['Low'], df['Close'])
        df = pd.concat([df, stoch], axis=1)

        # 取最新的技術指標數據
        latest = df.iloc[-1]

        analysis_result = {
            'RSI': round(latest['RSI'], 2),
            'MACD': round(latest['MACD_12_26_9'], 2),
            'MACD_Signal': round(latest['MACDs_12_26_9'], 2),
            'SMA_20': round(latest['SMA_20'], 2),
            'SMA_60': round(latest['SMA_60'], 2),
            'K值': round(latest['STOCHk_14_3_3'], 2),
            'D值': round(latest['STOCHd_14_3_3'], 2),
            '收盤價': round(latest['Close'], 2)
        }

        # 產生技術分析信號
        signals = generate_trading_signals(analysis_result)

        return {
            'indicators': analysis_result,
            'signals': signals,
            'historical_data': df.tail().to_dict('records')
        }

    except Exception as e:
        return f"無法進行技術分析: {str(e)}"

In [ ]:
def generate_trading_signals(data):
    signals = []

    # RSI 分析
    if data['RSI'] > 70:
        signals.append("RSI 顯示超買情況")
    elif data['RSI'] < 30:
        signals.append("RSI 顯示超賣情況")

    # MACD 分析
    if data['MACD'] > data['MACD_Signal']:
        signals.append("MACD 形成黃金交叉")
    elif data['MACD'] < data['MACD_Signal']:
        signals.append("MACD 形成死亡交叉")

    # KD 分析
    if data['K值'] < 20 and data['D值'] < 20:
        signals.append("KD 指標處於超賣區")
    elif data['K值'] > 80 and data['D值'] > 80:
        signals.append("KD 指標處於超買區")

    # 均線分析
    if data['SMA_20'] > data['SMA_60']:
        signals.append("20日均線突破60日均線")
    elif data['SMA_20'] < data['SMA_60']:
        signals.append("20日均線跌破60日均線")

    return signals

In [ ]:
get_technical_analysis('2330.TW')

In [ ]:
# 總管理 Agent 的指令
general_agent = Agent(
    name="總管理Agent",
    instructions="""你是股票市場查詢的總管理者。
    根據使用者的問題類型，將請求轉發給適當的專家：
    - 股價相關問題 -> 股價查詢專家
    - 財務數據相關問題 -> 財務分析專家
    - 技術指標相關問題 -> 技術分析專家
    如果使用者想要完整分析，可以依序調用多個專家的服務。""",
    functions=[get_stock_info, get_financial_data, get_technical_analysis]
)

# 股價查詢專家 Agent
price_agent = Agent(
    name="股價查詢專家",
    instructions="""你是專門處理股價查詢的專家。
    使用 get_stock_info 和 get_stock_price_history 函數來回答：
    - 即時股價
    - 歷史走勢
    - 52週高低點等資訊
    請用專業且易懂的方式解釋數據。""",
    functions=[get_stock_info, get_stock_price_history]
)

# 創建財務分析專家 Agent
financial_agent = Agent(
    name="財務分析專家",
    instructions="""你是專門分析財務數據的專家。
    使用 get_financial_data 函數來分析：
    - 基本財務指標
    - 獲利能力
    - 財務結構
    提供專業的財務分析建議。""",
    functions=[get_financial_data]
)

# 技術分析專家 Agent
technical_agent = Agent(
    name="技術分析專家",
    instructions="""你是專門進行技術分析的專家。
    使用 get_technical_analysis 函數來分析：
    - RSI、MACD、KD等技術指標
    - 移動平均線
    - 產生交易信號
    請用專業且易懂的方式解釋技術分析結果，並提供可能的交易建議。
    記得提醒使用者技術分析僅供參考，需要綜合其他因素做決定。""",
    functions=[get_technical_analysis]
)

# 創建 Swarm client
client = Swarm()



In [ ]:
print("歡迎使用股票市場助手！")
print("您可以詢問：")
print("1. 股票即時報價與走勢")
print("2. 基本財務分析")
print("3. 基本技術分析")
print("請輸入股票代號（例如：2330.TW）和您的問題。")

context_variables = {}
current_agent = general_agent

while True:
    user_input = input("\n您的問題: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("謝謝使用，再見！")
        break

    response = client.run(
        agent=current_agent,
        messages=[{"role": "user", "content": user_input}],
        context_variables=context_variables
    )

    print(f"\n{response.agent.name}:", response.messages[-1]["content"])
    current_agent = response.agent
    context_variables = response.context_variables
